In [1]:
import torch
from torch import nn, optim
import os 
import torch.nn.functional as F

In [2]:
from IPython.display import Audio
from IPython.utils import io
from synthesizer.inference import Synthesizer
from encoder import inference as encoder
from vocoder import inference as vocoder
from pathlib import Path
import numpy as np
import librosa
encoder_weights = Path("encoder/saved_models/pretrained.pt")
encoder.load_model(encoder_weights)
vocoder_weights = Path("vocoder/saved_models/pretrained/pretrained.pt")
syn_dir = Path("synthesizer/saved_models/logs-pretrained/taco_pretrained")
encoder.load_model(encoder_weights)
synthesizer = Synthesizer(syn_dir)
vocoder.load_model(vocoder_weights)

D:\pycharm_project\vs2ttsvoice\encoder\audio.py:13: UserWarning: Unable to import 'webrtcvad'. This package enables noise removal and is recommended.
  warn("Unable to import 'webrtcvad'. This package enables noise removal and is recommended.")


Loaded encoder "pretrained.pt" trained to step 1564501
Loaded encoder "pretrained.pt" trained to step 1564501
Found synthesizer "pretrained" trained to step 278000
Building Wave-RNN
Trainable Parameters: 4.481M
Loading model weights at vocoder\saved_models\pretrained\pretrained.pt


In [3]:
def embedEX(in_fpath):
    filepath=r'D:\voiceset\dataset\test_wav'
#     filepath=r'D:\voiceset\dataset\wav'
    in_fpath=os.path.join(filepath,in_fpath[:7],in_fpath[8:-6],in_fpath[-5:]+'.wav')
    reprocessed_wav = encoder.preprocess_wav(in_fpath)
    original_wav, sampling_rate = librosa.load(in_fpath)
    preprocessed_wav = encoder.preprocess_wav(original_wav, sampling_rate)
    embed,_ = encoder.embed_utterance(preprocessed_wav)
    return embed

In [4]:
class X2E_Net3L1L(torch.nn.Module):
    def __init__(self, in_dim, n_hidden_1,n_hidden_2,out_dim):
        super(X2E_Net3L1L, self).__init__()
        self.layer1 = torch.nn.Sequential(torch.nn.Linear(in_dim, n_hidden_1), torch.nn.ReLU(True))
        self.layer2 = torch.nn.Sequential(torch.nn.Linear(n_hidden_1, n_hidden_2), torch.nn.ReLU(True))  
        self.layer3 = torch.nn.Sequential(torch.nn.Linear(n_hidden_2, out_dim))
 
    def forward(self, x):
        x=x/torch.norm(x, p=2)
        x = self.layer1(x)
        x = self.layer2(x)
        x = self.layer3(x)
        x=x/torch.norm(x, p=2)
        return x

In [5]:
#模型
model=X2E_Net3L1L(512,400,300,256)
#损失函数
# criterion = torch.nn.MSELoss(reduce=True, size_average=False)
# criterion = torch.nn.MSELoss(reduction='sum')
criterion = torch.nn.L1Loss(reduction='sum')
#优化器
# optimizer = optim.SGD(model.parameters(), lr=learning_rate)
optimizer = torch.optim.Adam(model.parameters(), lr=0.001)

In [8]:
# 训练模型
epoch = 0
totaldir='D:\\pycharm_project\\vs2ttsvoice\\X2E'
#损失值写入文件
se_dir='D:\\pycharm_project\\vs2ttsvoice\\SE_vector'
for i in range(80):
    print(i)
    pathstr ='xvector'+str(i+1)+'.txt' 
    pathstr=os.path.join(totaldir,pathstr)
    sepath='dpdir'+str(i+1)+'.txt' 
    sepath=os.path.join(se_dir,sepath)
    se_vectors=open(sepath).readlines()
    file=open( pathstr)
    j=0
    for line in file.readlines():
        data=line.strip().replace('[','').replace(']','').split()
        se=se_vectors[j].replace('[','').replace(']','').split()
        se_filename=se[0]
        filename=data[0]
        if se_filename==filename:
#             print(se_filename)
            data=data[1:]
            se_data=se[1:]
            out=list(map(float,data))
            out=torch.tensor(out)
            pred=model(out)
            label=list(map(float,se_data))
            label=torch.tensor(label)
            loss = criterion(pred, label)
            print_loss = loss.data.item()
            optimizer.zero_grad()
            loss.backward()
            optimizer.step()
            epoch+=1
            if epoch%100 == 0:
            #         print('label:{0}\n,out:{1}\n,pred:{2}\n'.format(label.size(),out.size(),pred.size()))
            #保存参数、
                print('epoch: {}, loss: {:.4}'.format(epoch, loss.data.item()))
                state = { 'model':model.state_dict(), 'epoch':epoch }                       
                if not os.path.isdir('checkpointX2E'):
                     os.mkdir('checkpointX2E')    
                torch.save(state, '.\checkpointX2E\model3L1L_epoch_%d.ckpt' % (epoch))
                file_save = open('lossX2E3L1L.txt',mode='a')
                file_save.write('\n'+'epoch:'+str(epoch)+'  loss:'+str(loss.data.item()))
                file_save.close() 
        else:continue
        j=j+1

0
epoch: 100, loss: 5.845
epoch: 200, loss: 5.006
epoch: 300, loss: 3.344
epoch: 400, loss: 6.759
epoch: 500, loss: 6.772
epoch: 600, loss: 4.268
epoch: 700, loss: 4.459
epoch: 800, loss: 4.775
epoch: 900, loss: 5.104
epoch: 1000, loss: 5.887
epoch: 1100, loss: 7.158
epoch: 1200, loss: 6.344
epoch: 1300, loss: 6.393
epoch: 1400, loss: 6.753
epoch: 1500, loss: 5.857
epoch: 1600, loss: 3.704
epoch: 1700, loss: 8.023
epoch: 1800, loss: 5.387
1
epoch: 1900, loss: 5.431
epoch: 2000, loss: 5.578
epoch: 2100, loss: 4.017
epoch: 2200, loss: 7.475
epoch: 2300, loss: 3.527
epoch: 2400, loss: 5.736
epoch: 2500, loss: 3.743
epoch: 2600, loss: 5.257
epoch: 2700, loss: 3.08
epoch: 2800, loss: 8.553
epoch: 2900, loss: 3.095
epoch: 3000, loss: 5.912
epoch: 3100, loss: 3.802
epoch: 3200, loss: 5.76
epoch: 3300, loss: 7.768
epoch: 3400, loss: 5.932
epoch: 3500, loss: 6.868
epoch: 3600, loss: 4.798
epoch: 3700, loss: 5.804
2
epoch: 3800, loss: 8.038
epoch: 3900, loss: 6.159
epoch: 4000, loss: 6.924
epoch

epoch: 32000, loss: 4.353
epoch: 32100, loss: 6.015
epoch: 32200, loss: 5.796
epoch: 32300, loss: 5.216
epoch: 32400, loss: 4.945
epoch: 32500, loss: 6.088
epoch: 32600, loss: 4.669
18
epoch: 32700, loss: 5.188
epoch: 32800, loss: 3.655
epoch: 32900, loss: 4.818
epoch: 33000, loss: 5.879
epoch: 33100, loss: 3.74
epoch: 33200, loss: 6.213
epoch: 33300, loss: 5.45
epoch: 33400, loss: 5.426
epoch: 33500, loss: 4.755
epoch: 33600, loss: 5.29
epoch: 33700, loss: 5.676
epoch: 33800, loss: 5.064
epoch: 33900, loss: 5.437
epoch: 34000, loss: 3.785
epoch: 34100, loss: 3.765
epoch: 34200, loss: 4.665
epoch: 34300, loss: 5.131
epoch: 34400, loss: 6.189
19
epoch: 34500, loss: 6.045
epoch: 34600, loss: 4.08
epoch: 34700, loss: 5.901
epoch: 34800, loss: 6.065
epoch: 34900, loss: 5.03
epoch: 35000, loss: 5.61
epoch: 35100, loss: 6.456
epoch: 35200, loss: 5.68
epoch: 35300, loss: 4.239
epoch: 35400, loss: 5.184
epoch: 35500, loss: 5.118
epoch: 35600, loss: 4.812
epoch: 35700, loss: 5.267
epoch: 35800,

epoch: 63500, loss: 3.847
epoch: 63600, loss: 3.235
epoch: 63700, loss: 4.794
epoch: 63800, loss: 5.006
epoch: 63900, loss: 3.478
epoch: 64000, loss: 5.883
epoch: 64100, loss: 3.871
35
epoch: 64200, loss: 5.155
epoch: 64300, loss: 5.456
epoch: 64400, loss: 3.754
epoch: 64500, loss: 3.857
epoch: 64600, loss: 5.648
epoch: 64700, loss: 3.87
epoch: 64800, loss: 4.082
epoch: 64900, loss: 5.484
epoch: 65000, loss: 5.025
epoch: 65100, loss: 3.802
epoch: 65200, loss: 6.04
epoch: 65300, loss: 3.453
epoch: 65400, loss: 5.878
epoch: 65500, loss: 4.832
epoch: 65600, loss: 5.542
epoch: 65700, loss: 4.474
epoch: 65800, loss: 4.575
epoch: 65900, loss: 2.479
36
epoch: 66000, loss: 5.663
epoch: 66100, loss: 4.796
epoch: 66200, loss: 6.054
epoch: 66300, loss: 5.398
epoch: 66400, loss: 5.303
epoch: 66500, loss: 4.465
epoch: 66600, loss: 6.388
epoch: 66700, loss: 5.031
epoch: 66800, loss: 4.657
epoch: 66900, loss: 6.026
epoch: 67000, loss: 6.08
epoch: 67100, loss: 5.219
epoch: 67200, loss: 5.669
epoch: 67

epoch: 95000, loss: 5.567
epoch: 95100, loss: 4.56
epoch: 95200, loss: 3.653
epoch: 95300, loss: 3.144
epoch: 95400, loss: 5.085
epoch: 95500, loss: 5.003
epoch: 95600, loss: 3.904
epoch: 95700, loss: 4.976
53
epoch: 95800, loss: 4.139
epoch: 95900, loss: 4.916
epoch: 96000, loss: 3.233
epoch: 96100, loss: 4.436
epoch: 96200, loss: 5.163
epoch: 96300, loss: 3.983
epoch: 96400, loss: 3.981
epoch: 96500, loss: 4.452
epoch: 96600, loss: 4.097
epoch: 96700, loss: 5.143
epoch: 96800, loss: 4.984
epoch: 96900, loss: 3.99
epoch: 97000, loss: 4.089
epoch: 97100, loss: 4.68
epoch: 97200, loss: 2.936
epoch: 97300, loss: 4.727
epoch: 97400, loss: 4.365
epoch: 97500, loss: 3.508
54
epoch: 97600, loss: 3.994
epoch: 97700, loss: 5.634
epoch: 97800, loss: 4.539
epoch: 97900, loss: 5.243
epoch: 98000, loss: 5.507
epoch: 98100, loss: 3.24
epoch: 98200, loss: 4.991
epoch: 98300, loss: 5.014
epoch: 98400, loss: 4.839
epoch: 98500, loss: 4.094
epoch: 98600, loss: 5.546
epoch: 98700, loss: 4.487
epoch: 988

68
epoch: 125500, loss: 2.725
epoch: 125600, loss: 4.303
epoch: 125700, loss: 4.38
epoch: 125800, loss: 4.976
epoch: 125900, loss: 3.542
epoch: 126000, loss: 5.157
epoch: 126100, loss: 4.773
epoch: 126200, loss: 5.382
epoch: 126300, loss: 3.215
epoch: 126400, loss: 4.569
epoch: 126500, loss: 5.809
epoch: 126600, loss: 3.206
epoch: 126700, loss: 5.294
epoch: 126800, loss: 3.657
epoch: 126900, loss: 3.26
epoch: 127000, loss: 4.632
epoch: 127100, loss: 4.966
epoch: 127200, loss: 3.754
epoch: 127300, loss: 4.713
epoch: 127400, loss: 4.503
69
epoch: 127500, loss: 5.404
epoch: 127600, loss: 3.664
epoch: 127700, loss: 4.323
epoch: 127800, loss: 4.741
epoch: 127900, loss: 4.421
epoch: 128000, loss: 4.746
epoch: 128100, loss: 5.54
epoch: 128200, loss: 6.58
epoch: 128300, loss: 4.132
epoch: 128400, loss: 5.311
epoch: 128500, loss: 5.044
epoch: 128600, loss: 4.517
epoch: 128700, loss: 5.126
epoch: 128800, loss: 3.643
epoch: 128900, loss: 4.151
epoch: 129000, loss: 5.189
epoch: 129100, loss: 4.49


In [6]:
# 读取训练参数
# checkpoint = torch.load(r'C:\NLP\tts_rtvc\rtvc\checkpoint\model3_epoch_56300.ckpt')
checkpoint = torch.load(r'D:\pycharm_project\vs2ttsvoice\checkpointX2E\model3L1L_epoch_148600.ckpt')
model.load_state_dict(checkpoint['model'])

<All keys matched successfully>

In [ ]:
#合成语音
from IPython.utils import io
import time
epoch=0
text="This is being said in my own voice.The computer has learned to do an impression of me."
file=open('.\X2E\pv_xvector.txt')
filenamelabel=''
for line in file.readlines():
# line=file.readline()
    start_time=time.time()
    data=line.strip().replace('[','').replace(']','').split()
    filename=data[0]
    if (filenamelabel==filename[:7]):
        continue
    print(filename)
    data=data[1:]
#     out=[[list(map(float,data))]]
#     out=torch.tensor(out)
#     out=out.permute(0,2,1)
#     pred=model(out)
#     pred=pred.squeeze(0)
#     pred=pred.tolist()
#     print("pred",pred)
    out=list(map(float,data))
    out=torch.tensor(out)
    pred=model(out)
    pred=pred.tolist()
#     print("pred",pred)
#     print("Synthesizing 预测语音...")
    with io.capture_output() as captured:
      specs = synthesizer.synthesize_spectrograms([text], [pred])
    generated_wav = vocoder.infer_waveform(specs[0])
    generated_wav = np.pad(generated_wav, (0, synthesizer.sample_rate), mode="constant")
    display(Audio(generated_wav, rate=synthesizer.sample_rate))
    end_time=time.time()
    print('消耗时间：',end_time-start_time)
#     sf.write(filename[:7]+"A.wav", generated_wav, synthesizer.sample_rate)
    epoch =+1
    #原始声音合成
    print('&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&')
    label=embedEX(filename)
    print("Synthesizing 原始语音...")
    print('阈值：',np.linalg.norm(pred-label))
    with io.capture_output() as captured:
       specs = synthesizer.synthesize_spectrograms([text], [label])
    generated_wav = vocoder.infer_waveform(specs[0])
    generated_wav = np.pad(generated_wav, (0, synthesizer.sample_rate), mode="constant")
    display(Audio(generated_wav, rate=synthesizer.sample_rate))
    filenamelabel=filename[:7]
#     sf.write(filename[:7]+".wav", generated_wav, synthesizer.sample_rate)

id10270-5r0dWxy17C8-00001
Instructions for updating:
This class is equivalent as tf.keras.layers.LSTMCell, and will be replaced by that in Tensorflow 2.0.
Instructions for updating:
Use `tf.keras.layers.Conv1D` instead.
Instructions for updating:
Please use `layer.__call__` method instead.
Instructions for updating:
Use keras.layers.BatchNormalization instead.  In particular, `tf.control_dependencies(tf.GraphKeys.UPDATE_OPS)` should not be used (consult the `tf.keras.layers.batch_normalization` documentation).
Instructions for updating:
Use keras.layers.dropout instead.
Instructions for updating:
Please use `keras.layers.Bidirectional(keras.layers.RNN(cell))`, which is equivalent to this API
Instructions for updating:
Please use `keras.layers.RNN(cell)`, which is equivalent to this API
Instructions for updating:
Please use `layer.add_weight` method instead.
Instructions for updating:
Call initializer instance with the dtype argument instead of passing it to the constructor
Instructions

消耗时间： 30.87141990661621
&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&
Synthesizing 原始语音...
阈值： 0.5230894823086386


id10271-1gtz-CUIygI-00001


消耗时间： 24.473933219909668
&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&
Synthesizing 原始语音...
阈值： 0.5362843124618257


id10272-0qqQJVFKpbA-00001
